In [6]:
from dotenv import load_dotenv

load_dotenv()

True

In [7]:
from langchain_core.tools import tool

@tool
def get_recent_orders_by_status(user, status):
    """
    Fetches the three most recent orders for a given user with a specific status.
    
    Args:
    user (User): The user whose orders are to be retrieved.
    status (str): The status of the orders to be retrieved.

    Returns:
    QuerySet: A list of order objects.
    """

    return f"{user} 사용자의 {status} 상태 주문 조회"

In [8]:
from langchain.tools.render import render_text_description

tools = [get_recent_orders_by_status]
rendered_tools = render_text_description(tools)

In [9]:
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain.output_parsers import PydanticOutputParser

class ToolNecessity(BaseModel):
    necessity: str = Field(description="Determines if the use of the tool is necessary.")

    @validator("necessity")
    def validate_necessity(cls, value):
        if value not in ["yes", "no"]:
            raise ValueError("Necessity must be either 'yes' or 'no'.")
        return value
    
parser = PydanticOutputParser(pydantic_object=ToolNecessity)
parser

PydanticOutputParser(pydantic_object=<class '__main__.ToolNecessity'>)

In [12]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [11]:
from langchain_openai import OpenAI

model = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.0)

In [13]:
prompt_and_model = prompt | model
output = prompt_and_model.invoke({"query": "내 이름은 '나들'이야"})
parser.invoke(output)

OutputParserException: Failed to parse ToolNecessity from completion {"necessity": "I'm sorry, I am not sure what you are asking for. Can you please provide more information?"}. Got: 1 validation error for ToolNecessity
necessity
  Necessity must be either 'yes' or 'no'. (type=value_error)

In [16]:
from langchain_core.prompts import PromptTemplate

template = """
You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:

{rendered_tools}

사용자의 입력을 보고 위의 도구를 사용할 필요가 있을지 판단해줘.
아래 단계를 따라 도구 사용 여부를 판단해.
1. 
사용 가능한 도구와 각 도구에 대한 설명을 빠짐없이 파악해.
2. 
사용자 입력에 대한 응답을 생성하는 데 필요한 도구가 있는지 파악해.
응답 생성에 도구가 필요하지 않다면 'no'라고 응답해.
응답 생성에 도구가 필요하고, 그 도구가 사용 가능한 도구 모음에 있다면 'yes'라고 응답해.
3.
이전 대화 또는 사용자 입력에서 도구 호출에 필요한 인자 모두를 추출할 수 있는지 확인해.
만약 필요 인자 모두를 추출하지 못한다면 'no'라고 응답해.

[!주의사항]
응답은 반드시 'yes' 또는 'no' 중 하나로 해야해.
응답을 출력하기 전에 대답이 'yes' 또는 'no' 중 하나인지 확인하고 만약 이 두 값 중 하나가 아닐 경우 다시 앞의 단계를 따라 판단한 후 두 값 중 하나로 답변해줘.

\n{format_instructions}\n{query}\n
"""
prompt = PromptTemplate(
    template=template ,
    input_variables=["query", "rendered_tools"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
prompt 

PromptTemplate(input_variables=['query', 'rendered_tools'], partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"necessity": {"title": "Necessity", "description": "Determines if the use of the tool is necessary.", "type": "string"}}, "required": ["necessity"]}\n```'}, template="\nYou are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:\n\n{rendered_tools}\n\n사용자의 입력을 보고 위의 도구를 사용할 필요가 있을지 판단해줘.\n아래 단계를 따라 도구 사용 여부를 판단해.\n1. \n사용 가능한 도구와 각 도구에 대한 설명을 빠짐없이 파악해.\n2. \n사용자

In [17]:
prompt_and_model = prompt | model
output = prompt_and_model.invoke({"query": "내 이름은 '나들'이야",
                                  "rendered_tools": rendered_tools})
parser.invoke(output)

OutputParserException: Failed to parse ToolNecessity from completion {"properties": {"necessity": {"title": "Necessity", "description": "Determines if the use of the tool is necessary.", "type": "string"}}, "required": ["necessity"]}. Got: 1 validation error for ToolNecessity
necessity
  field required (type=value_error.missing)

In [18]:
output

'\n{\n    "properties": {\n        "necessity": {\n            "title": "Necessity",\n            "description": "Determines if the use of the tool is necessary.",\n            "type": "string"\n        }\n    },\n    "required": ["necessity"]\n}\n\n1. 사용 가능한 도구와 각 도구에 대한 설명을 빠짐없이 파악해.\n- get_recent_orders_by_status: 사용자의 최근 3개 주문을 특정 상태로 가져오는 도구입니다.\n\n2. 사용자 입력에 대한 응답을 생성하는 데 필요한 도구가 있는지 파악해.\n- 사용자 입력에 따라 최근 주문을 가져오는 도구가 필요합니다.\n\n3. 이전 대화 또는 사용자 입력에서 도구 호출에 필요한 인자 모두를 추출할 수 있는지 확인해.\n- 사용자와 상태에 대한 인자가 필요합니다.'

---

In [25]:
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import OpenAI

model = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.0)


# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field


# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

prompt_and_model = prompt | model

In [26]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"setup": {"title": "Setup", "description": "question to set up a joke", "type": "string"}, "punchline": {"title": "Punchline", "description": "answer to resolve the joke", "type": "string"}}, "required": ["setup", "punchline"]}\n```'

In [21]:
output = prompt_and_model.invoke({"query": "Tell me a joke."})
output

'\n{\n    "setup": "Why did the tomato turn red?",\n    "punchline": "Because it saw the salad dressing!"\n}'

In [22]:
parser.invoke(output)

Joke(setup='Why did the tomato turn red?', punchline='Because it saw the salad dressing!')

---

In [34]:
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain.output_parsers import PydanticOutputParser

class ToolNecessity(BaseModel):
    necessity: str = Field(description="Determines if the use of the tool is necessary.")

    @validator("necessity")
    def validate_necessity(cls, value):
        if value not in ["yes", "no"]:
            raise ValueError("Necessity must be either 'yes' or 'no'.")
        return value
    
parser = PydanticOutputParser(pydantic_object=ToolNecessity)
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"necessity": {"title": "Necessity", "description": "Determines if the use of the tool is necessary.", "type": "string"}}, "required": ["necessity"]}\n```'

In [27]:
rendered_tools

'get_recent_orders_by_status: get_recent_orders_by_status(user, status) - Fetches the three most recent orders for a given user with a specific status.\n\nArgs:\nuser (User): The user whose orders are to be retrieved.\nstatus (str): The status of the orders to be retrieved.\n\nReturns:\nQuerySet: A list of order objects.'

In [35]:
from langchain_core.prompts import PromptTemplate

template = """
You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:

{rendered_tools}

사용자의 입력을 보고 위의 도구를 사용할 필요가 있을지 판단해줘.
아래 단계를 따라 도구 사용 여부를 판단해.
1. 
사용 가능한 도구와 각 도구에 대한 설명을 빠짐없이 파악해.
2. 
사용자 입력에 대한 응답을 생성하는 데 필요한 도구가 있는지 파악해.
응답 생성에 도구가 필요하지 않다면 'no'라고 응답해.
응답 생성에 도구가 필요하고, 그 도구가 사용 가능한 도구 모음에 있다면 'yes'라고 응답해.
3.
이전 대화 또는 사용자 입력에서 도구 호출에 필요한 인자 모두를 추출할 수 있는지 확인해.
만약 필요 인자 모두를 추출하지 못한다면 'no'라고 응답해.

[!주의사항]
응답은 반드시 'yes' 또는 'no' 중 하나로 해야해.
응답을 출력하기 전에 대답이 'yes' 또는 'no' 중 하나인지 확인하고 만약 이 두 값 중 하나가 아닐 경우 다시 앞의 단계를 따라 판단한 후 두 값 중 하나로 답변해줘.

\n{format_instructions}\n{query}\n
"""
prompt = PromptTemplate(
    template=template ,
    input_variables=["query", "rendered_tools"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
prompt

PromptTemplate(input_variables=['query', 'rendered_tools'], partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"necessity": {"title": "Necessity", "description": "Determines if the use of the tool is necessary.", "type": "string"}}, "required": ["necessity"]}\n```'}, template="\nYou are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:\n\n{rendered_tools}\n\n사용자의 입력을 보고 위의 도구를 사용할 필요가 있을지 판단해줘.\n아래 단계를 따라 도구 사용 여부를 판단해.\n1. \n사용 가능한 도구와 각 도구에 대한 설명을 빠짐없이 파악해.\n2. \n사용자

In [36]:
prompt_and_model = prompt | model
output = prompt_and_model.invoke({"query": "내 이름은 '나들'이야",
                                  "rendered_tools": rendered_tools})
output

'\n{\n    "properties": {\n        "necessity": {\n            "title": "Necessity",\n            "description": "Determines if the use of the tool is necessary.",\n            "type": "string"\n        }\n    },\n    "required": ["necessity"]\n}\n\n1. 사용 가능한 도구와 각 도구에 대한 설명을 빠짐없이 파악해.\n- get_recent_orders_by_status: 사용자의 최근 3개 주문을 특정 상태로 가져오는 도구입니다.\n\n2. 사용자 입력에 대한 응답을 생성하는 데 필요한 도구가 있는지 파악해.\n- 사용자 입력에 따라 최근 주문을 가져오는 도구가 필요합니다.\n\n3. 이전 대화 또는 사용자 입력에서 도구 호출에 필요한 인자 모두를 추출할 수 있는지 확인해.\n- 사용자와 상태에 대한 인자가 필요합니다.'

In [37]:
from langchain_core.prompts import PromptTemplate

template = """
You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:

{rendered_tools}

사용자의 입력을 보고 위의 도구를 사용할 필요가 있을지 판단해줘.
아래 단계를 따라 도구 사용 여부를 판단해.
1. 
사용 가능한 도구와 각 도구에 대한 설명을 빠짐없이 파악해.
2. 
사용자 입력에 대한 응답을 생성하는 데 필요한 도구가 있는지 파악해.
응답 생성에 도구가 필요하지 않다면 necessity 필드값은 'no'가 되어야 해.
응답 생성에 도구가 필요하고, 그 도구가 사용 가능한 도구 모음에 있다면 necessity 필드값은 'yes'가 되어야 해.
3.
이전 대화 또는 사용자 입력에서 도구 호출에 필요한 인자 모두를 추출할 수 있는지 확인해.
만약 필요 인자 모두를 추출하지 못한다면 necessity 필드값은 'no'가 되어야 해.

[!주의사항]
necessity 필드값은 반드시 'yes' 또는 'no' 중 하나로 해야해.
응답을 출력하기 전에 necessity 필드값이 'yes' 또는 'no' 중 하나인지 확인하고 만약 이 두 값 중 하나가 아닐 경우 다시 앞의 단계를 따라 판단한 후 두 값 중 하나로 답변해줘.

\n{format_instructions}\n{query}\n
"""
prompt = PromptTemplate(
    template=template ,
    input_variables=["query", "rendered_tools"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
prompt

PromptTemplate(input_variables=['query', 'rendered_tools'], partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"necessity": {"title": "Necessity", "description": "Determines if the use of the tool is necessary.", "type": "string"}}, "required": ["necessity"]}\n```'}, template="\nYou are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:\n\n{rendered_tools}\n\n사용자의 입력을 보고 위의 도구를 사용할 필요가 있을지 판단해줘.\n아래 단계를 따라 도구 사용 여부를 판단해.\n1. \n사용 가능한 도구와 각 도구에 대한 설명을 빠짐없이 파악해.\n2. \n사용자

In [38]:
prompt_and_model = prompt | model
output = prompt_and_model.invoke({"query": "내 이름은 '나들'이야",
                                  "rendered_tools": rendered_tools})
output

'\n{\n    "properties": {\n        "necessity": {\n            "title": "Necessity",\n            "description": "Determines if the use of the tool is necessary.",\n            "type": "string"\n        }\n    },\n    "required": ["necessity"]\n}\n\n1. 사용 가능한 도구와 각 도구에 대한 설명을 빠짐없이 파악해.\n- get_recent_orders_by_status: 사용자의 최근 주문을 가져오는 도구로, 사용자와 주문 상태를 인자로 받아 QuerySet 형태로 주문 객체를 반환한다.\n\n2. 사용자 입력에 대한 응답을 생성하는 데 필요한 도구가 있는지 파악해.\n- 사용자 입력에 따라 최근 주문 상태를 확인해야 할 경우, get_recent_orders_by_status 도구가 필요하다. 따라서 necessity 필드값은 \'yes\'가 된다.\n\n3. 이전 대화 또는 사용자 입력에서 도구 호출에 필요한 인자 모두를 추'

---

In [46]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = f"""
You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:
Given the user input, determine whether there is a necessity to use the tool and respond with yes or no. Return your response as a JSON blob with the 'necessity' key.

{rendered_tools}

사용자의 입력을 보고 위의 도구를 사용할 필요가 있을지 판단해줘.
아래 단계를 따라 도구 사용 여부를 판단해.
1. 
사용 가능한 도구와 각 도구에 대한 설명을 빠짐없이 파악해.
2. 
사용자 입력에 대한 응답을 생성하는 데 필요한 도구가 있는지 파악해.
응답 생성에 도구가 필요하지 않다면 necessity 필드값은 'no'가 되어야 해.
응답 생성에 도구가 필요하고, 그 도구가 사용 가능한 도구 모음에 있다면 necessity 필드값은 'yes'가 되어야 해.
3.
이전 대화 또는 사용자 입력에서 도구 호출에 필요한 인자 모두를 추출할 수 있는지 확인해.
만약 필요 인자 모두를 추출하지 못한다면 necessity 필드값은 'no'가 되어야 해.

[!주의사항]
- necessity 필드값은 반드시 'yes' 또는 'no' 중 하나로 해야해.
  응답을 출력하기 전에 necessity 필드값이 'yes' 또는 'no' 중 하나인지 확인하고 만약 이 두 값 중 하나가 아닐 경우 다시 앞의 단계를 따라 판단한 후 두 값 중 하나로 답변해줘.
- 도구 호출에 필요한 인자 모두를 빠짐없이 추출할 수 있는지 확실히 파악해야 해.
"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

In [47]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")
model

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000281871B0FA0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000028187187B80>, openai_api_key=SecretStr('**********'), openai_proxy='')

In [48]:
base_chain = prompt | model 
base_chain

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\nYou are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:\nGiven the user input, determine whether there is a necessity to use the tool and respond with yes or no. Return your response as a JSON blob with the 'necessity' key.\n\nget_recent_orders_by_status: get_recent_orders_by_status(user, status) - Fetches the three most recent orders for a given user with a specific status.\n\nArgs:\nuser (User): The user whose orders are to be retrieved.\nstatus (str): The status of the orders to be retrieved.\n\nReturns:\nQuerySet: A list of order objects.\n\n사용자의 입력을 보고 위의 도구를 사용할 필요가 있을지 판단해줘.\n아래 단계를 따라 도구 사용 여부를 판단해.\n1. \n사용 가능한 도구와 각 도구에 대한 설명을 빠짐없이 파악해.\n2. \n사용자 입력에 대한 응답을 생성하는 데 필요한 도구가 있는지 파악해.\n응답 생성에 도구가 필요하지 않다면 necessity 필드값은 'no'가 되어야 해.\n응답 생성에 도구가 필요하고, 그 도구가 사용 가능한 도구 모음에 있다면 necessit

In [49]:
response = base_chain.invoke(
    {"input": "내 이름은 젖수야"}
)
print(response.content)

{
  "necessity": "no"
}


In [50]:
response = base_chain.invoke(
    {"input": "주문 조회를 하고 싶어"}
)
print(response.content)

{
  "necessity": "no"
}


In [51]:
response = base_chain.invoke(
    {"input": "주문 상태가 주문 완료인 모든 주문 내역을 보고 싶어"}
)
print(response.content)

{
  "necessity": "yes"
}


In [52]:
response = base_chain.invoke(
    {"input": "주문 상태가 배송 완료인 모든 주문 내역을 보고 싶어"}
)
print(response.content)

{
  "necessity": "yes"
}


In [53]:
model = ChatOpenAI(model="gpt-4")
base_chain = prompt | model 

In [54]:
response = base_chain.invoke(
    {"input": "주문 상태가 배송 완료인 모든 주문 내역을 보고 싶어"}
)
print(response.content)

{"necessity": "yes"}
